## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...
1,2025-08-10-01-00-00 +0000,wsj,Just as Russia’s Most-Famous Dissident Seemed ...,https://www.wsj.com/world/russia/navalny-death...
2,2025-08-10-00-41-44 +0000,bbc,How Kentucky bourbon went from boom to bust,https://www.bbc.com/news/articles/ckglnk6yxlko...
3,2025-08-10-00-36-08 +0000,startribune,Scoggins: J.J. McCarthy feeling ‘super clean’ ...,https://www.startribune.com/minnesota-vikings-...
4,2025-08-10-00-28-29 +0000,bbc,Billionaire inheritance feud spotlights India'...,https://www.bbc.com/news/articles/c6203djnp84o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2445/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,34
156,new,11
107,putin,11
325,ukraine,10
23,russia,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
65,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...,89
118,2025-08-09-09-48-43 +0000,nypost,Zelensky vows Russia will not get any Ukrainia...,https://nypost.com/2025/08/09/us-news/zelensky...,80
0,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...,70
10,2025-08-09-23-34-20 +0000,nypost,White House weighs inviting Zelensky to Trump’...,https://nypost.com/2025/08/09/us-news/white-ho...,68
76,2025-08-09-16-14-54 +0000,bbc,Putin gives Trump envoy award for CIA official...,https://www.bbc.com/news/articles/c04rd19e5epo...,68


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
65,89,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...
0,36,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...
26,35,2025-08-09-22-47-00 +0000,wsj,Adriana Kugler hasn’t said why she is leaving ...,https://www.wsj.com/economy/central-banking/su...
42,32,2025-08-09-20-17-00 +0000,bbc,Atlanta police officer dies after shooting nea...,https://www.bbc.com/news/articles/cp3ekez0k2xo...
118,31,2025-08-09-09-48-43 +0000,nypost,Zelensky vows Russia will not get any Ukrainia...,https://nypost.com/2025/08/09/us-news/zelensky...
143,30,2025-08-09-08-00-00 +0000,cbc,"Deal or no deal, Mark Carney has to manage a n...",https://www.cbc.ca/news/politics/carney-tariff...
69,30,2025-08-09-17-07-56 +0000,nypost,Financial expert warns young Americans against...,https://nypost.com/2025/08/09/business/financi...
36,25,2025-08-09-21-07-54 +0000,bbc,Trump nominates ex-Fox News host Tammy Bruce a...,https://www.bbc.com/news/articles/cj9wv49xr0xo...
116,24,2025-08-09-10-00-00 +0000,latimes,SpaceX hopes to nearly double the rocket launc...,https://www.latimes.com/california/story/2025-...
33,23,2025-08-09-21-47-06 +0000,nypost,"US, Qatari mediators try to shape last-minute ...",https://nypost.com/2025/08/09/world-news/us-qa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
